In [ ]:
import json
import os
from pathlib import Path
import torch
from pyannote.audio import Pipeline

with open("secrets.json") as f:
    secrets = json.load(f)
os.environ["CUDA_VISIBLE_DEVICE"] = "6"
# Load the pipeline:
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1", use_auth_token=secrets["hf_token"]
).to(torch.device("cuda"))


files = [
    file.with_suffix("").name
    for file in Path("data/audio_16khz_mono_wav/").glob("*.wav")
]

diarization = pipeline(
    "/cache/peterr/ASR_pipeline/data/audio_input/Iriss-J-Gvecg-P500048-avd.wav",
    #  min_speakers=2, max_speakers=4
)


In [ ]:
%matplotlib inline
diarization

In [ ]:
file

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

# create the Annotation instance
from pyannote.core import Annotation, Segment
annotation = Annotation()
annotation[Segment(2, 5)] = 'A'
annotation[Segment(4, 8)] = 'B'
annotation[Segment(9, 10)] = 'A'
from pyannote.core import notebook
notebook.crop = Segment(0, 200)

# plot
figure, ax = plt.subplots(figsize=(7, 3))
notebook.plot_annotation(diarization, ax=ax, time=True, legend=True)

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"


from transformers import AutoProcessor, SeamlessM4Tv2ForSpeechToText
import torchaudio
import torch

processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model = SeamlessM4Tv2ForSpeechToText.from_pretrained("facebook/seamless-m4t-v2-large", )


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: 'module' object is not callable

In [5]:


# from audio
audio, orig_freq =  torchaudio.load("/cache/peterr/ASR_pipeline/1_beginning.wav")
audio =  torchaudio.functional.resample(audio, orig_freq=orig_freq, new_freq=16_000) # must be a 16 kHz waveform array
audio_inputs = processor(audios=audio, return_tensors="pt",)
output = model.generate(**audio_inputs,
                    # generate_speech=False,
                    tgt_lang="slv",
                    # return_intermediate_token_ids=True,
                    
                    )

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


ValueError: The following `model_kwargs` are not used by the model: ['device'] (note: typos in the generate arguments will also show up in this list)

In [4]:
output

tensor([[     3, 256077,    530, 247681,  19690,   1189,   7144,   2399,    572,
          85673, 247681,   2917,   1189,   7144,  11608,  12299,    377, 247692,
           1156,    452,  41015, 247676,      3]])

In [13]:
processor.tokenizer.batch_decode(output)[0].replace("</s>", "").replace("__slv__", "")

' ja, jaz sem bila presenečena, ker sem bila še zmeraj utrujena.'